In [ ]:
pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# the following two import statements provide the implemetation for Selenium Waits
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import pandas as pd

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

In [ ]:
# import pandas and plotly
 
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html, dcc 
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
from urllib.request import urlopen
# run Dash apps from within Jupyter Notebook environments
from jupyter_dash import JupyterDash
# imports
from dash import html, dcc, Dash
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:

house_link = "https://www.politico.com/2022-election/results/"
driver.get(house_link)


In [ ]:

# Get the page source
page_source = driver.page_source

# Parse the page source using Beautiful Soup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table that contains the list of states
container = soup.find('div','styles_columns-container__Kq5vS')

# Extract the list of states from the table
states = [state.get_text(strip=True) for state in container.find_all('a')]

# Print the list of states
print(states)


In [ ]:
states_link = [state.replace(' ', '-') for state in states]

In [ ]:
print (states_link)

In [ ]:
import pandas

df = pd.DataFrame(columns=['Candidate', 'Party', 'Votes', 'Pct%','State', 'District'])

In [ ]:
df

In [ ]:
#this code spends lot of time to run
count=0
final_results = {}

for index, link in enumerate(states_link):
    print(index, 'Loading Data for', states[index])
    result = {}
    driver.get(house_link + str(link.lower())+'/house/')
    try:
        buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[title="Expand"]')))
        buttons = driver.find_elements(By.CSS_SELECTOR, '[title="Expand"]')
        print(' -----',len(buttons), 'button found.')
        for button in buttons:
            button.click() 
            print(states[index], button.text, 'clicked')
        buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[title="Expand"]')))
        buttons = driver.find_elements(By.CSS_SELECTOR, '[title="Expand"]')
        print(' -----',len(buttons), 'button found.')
        for button in buttons:
            button.click() 
            print(states[index], button.text, 'clicked')
    except:
        print(states[index],'no button found')
    # start scraping districts
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #print(soup)

    container = soup.find('div',{'class':'styles_table-container__vTHda'})

    districts = container.find_all('h5')

    districts = [district.get_text(strip=True) for district in districts]


    #scrapping candidate
    sections = []
    sub_list = []
    if container.find_all('div',{'class':'styles_left-title__rNUfI'}):
        sections = container.find_all('div',{'class':'styles_left-title__rNUfI'})
    
    
    if sections != []:
        
        for section_index, section in enumerate(sections):
            candidates = section.find_all('td')
            new_list = []

            list1 = [candidate.get_text(strip=True) for candidate in candidates]
            for item in list1:
                if '(' in item:
                    x = item.split('(')
                    for y in x:
                        new_list.append(y)
            
                elif item =='':
                    new_list.append('0')
                    new_list.append('0%')
                elif item.replace(',','').isdigit() == False and '%' not in item and item!= 'Uncontested' and item !='':
                    new_list.append(item)
                    new_list.append('Other Party')
                elif 'Uncontested' == item:
                    next
                
                else: new_list.append(item)
            
            N = 4
            temp_list = [new_list[n:n+N] for n in range(0,len(new_list), N)]
            for sublist in temp_list:
                sublist.append(states[index])
                sublist.append(districts[section_index])
            sub_list.extend(temp_list)
            

                    
    else: 
        candidates = container.find_all('td')
        list1 = [candidate.get_text(strip=True) for candidate in candidates]
        new_list = []
        for item in list1:
            if '(' in item:
                x = item.split('(')
                for y in x:
                    new_list.append(y)
            elif item =='':
                new_list.append('0')
                new_list.append('0.00%')
            elif item.replace(',','').isdigit() == False and '%' not in item and item!= 'Uncontested' and item !='':
                new_list.append(item)
                new_list.append('Other Party')
            elif 'Uncontested' == item:
                next
                
            else: new_list.append(item)
        
        N=4
        
        sub_list = [new_list[n:n+N] for n in range(0,len(new_list), N)]
        for list in sub_list:
            list.append(states[index])
            list.append('At Large')

    new_df = pd.DataFrame(sub_list, columns = df.columns)
    df = pd.concat([df, new_df])
    

    
    count += 1
    

print(count , 'states scraped')

    

In [ ]:
df

In [ ]:
df = df[df['Candidate'] != 'Hide other candidates']

In [ ]:
incumbent = ['Incumbent' if '*' in candidate else '/' for candidate in df['Candidate']]
df['Incumbent'] = incumbent

In [ ]:
df['Party'] = df['Party'].replace('\)','',regex=True)

In [ ]:
df

In [ ]:
party_dict = {'R': 'Republician', 'D': 'Democratic', 'Ind.': 'Independent Party'}
df['Party'] = df['Party'].replace(party_dict)

In [ ]:
us_state_to_code = {
            "Alabama": "AL",
            "Alaska": "AK",
            "Arizona": "AZ",
            "Arkansas": "AR",
            "California": "CA",
            "Colorado": "CO",
            "Connecticut": "CT",
            "Delaware": "DE",
            "Florida": "FL",
            "Georgia": "GA",
            "Hawaii": "HI",
            "Idaho": "ID",
            "Illinois": "IL",
            "Indiana": "IN",
            "Iowa": "IA",
            "Kansas": "KS",
            "Kentucky": "KY",
            "Louisiana": "LA",
            "Maine": "ME",
            "Maryland": "MD",
            "Massachusetts": "MA",
            "Michigan": "MI",
            "Minnesota": "MN",
            "Mississippi": "MS",
            "Missouri": "MO",
            "Montana": "MT",
            "Nebraska": "NE",
            "Nevada": "NV",
            "New Hampshire": "NH",
            "New Jersey": "NJ",
            "New Mexico": "NM",
            "New York": "NY",
            "North Carolina": "NC",
            "North Dakota": "ND",
            "Ohio": "OH",
            "Oklahoma": "OK",
            "Oregon": "OR",
            "Pennsylvania": "PA",
            "Rhode Island": "RI",
            "South Carolina": "SC",
            "South Dakota": "SD",
            "Tennessee": "TN",
            "Texas": "TX",
            "Utah": "UT",
            "Vermont": "VT",
            "Virginia": "VA",
            "Washington": "WA",
            "West Virginia": "WV",
            "Wisconsin": "WI",
            "Wyoming": "WY",
            "District of Columbia": "DC",
            "American Samoa": "AS",
            "Guam": "GU",
            "Northern Mariana Islands": "MP",
            "Puerto Rico": "PR",
            "United States Minor Outlying Islands": "UM",
            "U.S. Virgin Islands": "VI",
        }

df['State Code'] = df['State'].map(us_state_to_code)
df

In [ ]:
df['Candidate'] = df['Candidate'].replace({'\*': ''}, regex=True)

In [ ]:
df['Votes'] = df['Votes'].replace({',': ''}, regex=True).astype(int)

In [ ]:
df

In [ ]:
df = df.loc[:, ['State', 'State Code', 'District', 'Party', 'Candidate', 'Incumbent', 'Votes', 'Pct%']]

In [ ]:
df['Elected'] = df.groupby(['State', 'District'])['Votes'].transform(max) == df['Votes']

In [ ]:
df

In [ ]:
party = df['Party'].tolist()

In [ ]:
won = []
for index, item in enumerate(party):
    get_index= index
    won.append(party[index])

In [ ]:
len(won)

In [ ]:
df['Won Party'] = won

In [ ]:
df

In [ ]:
df.to_csv('final_house.csv', index=False)

In [ ]:
#Part2: Building the dash apps
# install the needed packages on colab
! pip install dash jupyter-dash --quiet

In [ ]:
df = pd.read_csv('final_house.csv')


In [ ]:
# keep the first duplicated result and drop the special election
df = df.drop_duplicates(subset=['State', 'District','Candidate'], keep='first')

In [ ]:
#Replace incumbent column with 'Yes' or 'No'
df['Incumbent'] = df['Incumbent'].replace({'/': 'No', 'Incumbent': 'Yes'})

In [ ]:
#Total seats
df_plot = df[df['Elected'] == True]
df_plot
df_repu = df[(df['Elected'] == True) & (df['Won Party'] == 'Republician')]
df_demo = df[(df['Elected'] == True) & (df['Won Party'] == 'Democratic')]
df[df['State'] == 'Alabama']

In [ ]:
#creating the dataframe for map
df_grouped = df.groupby('State')

state_code = df_grouped.apply(lambda x: x['State Code'].iloc[0])
Won_party = df_grouped.apply(lambda x: x.loc[x['Elected'], 'Won Party'].value_counts().index[0])

rep_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Republician')).sum())
demo_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Democratic')).sum())
lib_num_win_seats = df_grouped.apply(lambda x: (x['Elected'] & (x['Won Party'] == 'Libertarian Party')).sum())

num_win_seats = df_grouped.apply(lambda x: max(
    sum(x['Elected'] & (x['Won Party'] == 'Republician')),
    sum(x['Elected'] & (x['Won Party'] == 'Democratic')),
    sum(x['Elected'] & (x['Won Party'] == 'Libertarian Party'))
))

total_seats = df_grouped['District'].nunique()

repu_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Republician', 'Votes'].sum())
demo_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Democratic', 'Votes'].sum())
liber_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Libertarian Party', 'Votes'].sum())
other_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Other Party', 'Votes'].sum())
green_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Green Party', 'Votes'].sum())
ind_votes_total = df_grouped.apply(lambda x: x.loc[x['Party'] == 'Independent Party', 'Votes'].sum())

repu_win_rate = rep_num_win_seats / total_seats

final_df = pd.DataFrame({
    'State': Won_party.index,
    'State Code': state_code.values,
    'Won Party': Won_party.values,
    'Won Seats': num_win_seats.values,
    'Total Seats': total_seats.values,
    'Republican Votes': repu_votes_total.values,
    'Democratic Votes': demo_votes_total.values,
    'Libertarian Party Votes': liber_votes_total.values,
    'Green Party Votes': green_votes_total.values,
    'Independent Party Votes': ind_votes_total.values,
    'Other Party Votes': other_votes_total.values,
    'Rep won seats %': repu_win_rate.values,
})

final_df['Rep won seats %'] = final_df['Rep won seats %'] *100
final_df

In [ ]:
#datasets for bar chart1
partyvotes_df = df.groupby(['State','State Code','Party'])['Votes'].sum()
partyvotes_df = partyvotes_df.reset_index()
partyvotes_df = partyvotes_df.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
partyvotes_df['color'] = partyvotes_df['Party']
partyvotes_df = partyvotes_df.rename(columns={'Party':'Parties'})
partyvotes_df

In [ ]:
#datasets for bar chart2
dff = df[df['Elected'] == True]
districtvotes_df = dff.groupby(['State','State Code','District'])['Votes'].sum()
districtvotes_df = districtvotes_df.reset_index()
districtvotes_df = pd.merge(districtvotes_df,dff[['State','District','Party','Candidate', 'Incumbent']],on=['State','District'],how='left')
districtvotes_df = districtvotes_df.drop_duplicates()
districtvotes_df

In [ ]:

fig = px.choropleth(final_df,locations='State Code',locationmode='USA-states',color='Rep won seats %',
                    hover_name='State',hover_data=['State Code','Won Party','Won Seats',
                                                   'Total Seats','Republican Votes','Democratic Votes',
                                                   'Libertarian Party Votes','Independent Party Votes','Green Party Votes','Other Party Votes','Rep won seats %'],
                    color_continuous_scale='RdBu_r', range_color=[0,100],scope='usa',projection = 'albers usa')
fig.update_layout(height=600,width=800)

In [ ]:
#Extra features: showing candidate with highest votes in each state
win_dff = df[df['Elected'] == True]
win_dff.loc[win_dff['State'] == 'Missouri', 'Candidate'] = 'Missouri Smith'
win_dff.loc[win_dff['State'] == 'Nebraska', 'Candidate'] = 'Nebraska Smith'
win_dff.loc[win_dff['State'] == 'New Jersey', 'Candidate'] = 'New Jersey Smith'
win_dff_grouped = win_dff.groupby('State').apply(lambda x: x.loc[x['Votes'].idxmax()])
win_dff_party = win_dff_grouped[['State','Candidate', 'Party', 'Votes']]
bar_chart3 = px.bar(win_dff_party, x='Candidate', y='Votes',hover_name='Candidate',hover_data=['State'],color= 'Party',
                    color_discrete_map={'Republician': 'rgb(178,24,43)','Democratic': 'rgb(33,102,172)'},title='The Candidate With Highest Votes In Each State')
bar_chart3

In [ ]:
#Extra feature showing all candidates in the district
#datasets for bar chart4
dff4 = df
dff4 = dff4.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
dff4

In [ ]:
#program without extra feature
# instantiate a JupyterDash object instead of a Dash object
app = JupyterDash(__name__)
app.layout = html.Div([html.Div([html.H1(children='US House Election 2022', id="heading"),
                       html.Hr()]),
                      html.Div([html.P(children=f'Total seats: {len(df_plot)} | Republician won: {len(df_repu)} | Democratic won: {len(df_demo)}',
                             style={'font-weight': 'bold'}),
                                dcc.Graph(id='us-election-2022-fig', figure=fig,clickData={'points': [{'customdata': ['AL']}]})],
                               style={'width': '70%', 'display': 'inline-block', 'padding': '0 20', 'float': 'left'}),
                       html.Div([html.P(id ='winning_statement',style={'font-weight': 'bold','font-size': '13px'}),dcc.Graph(id='winning-party-votes',clickData={'points': [{'customdata': ['AL']}]}),dcc.Graph(id='winning-districts-votes')],
                               style={'width': '29%', 'display': 'inline-block', 'padding': '0 20', 'float': 'right'})
                       
                        ])


@app.callback(Output('winning_statement','children'),
              Output('winning-party-votes','figure'),
              Output('winning-districts-votes','figure'),
              Input('us-election-2022-fig', 'clickData')) 
def UsElection(received):
    if not received: 
        raise PreventUpdate
    received_data = received['points'][0]['customdata'][0]
    won_parties = final_df.loc[final_df['State Code'] == received_data,'Won Party'].values[0]
    win_statement = f'Winning Party: {won_parties}'
    dff = partyvotes_df[partyvotes_df['State Code'] == received_data]
    required_state = df.loc[df['State Code'] == received_data, 'State'].values[0]
    bar_chart = px.bar(dff, x='Parties',
                         y='Votes',title= f'State: {required_state}',color= 'Parties',hover_data=['color'],
                       color_discrete_map={
        'Republician': 'rgb(178,24,43)',
        'Democratic': 'rgb(33,102,172)',
        'Libertarian': 'rgb(253,219,199)',
        'Green': 'rgb (0,128,0)',
        'Independent': 'rgb(255,255,0)',
    },category_orders={'Parties':['Republician','Democratic']})
    bar_chart.update_layout(height=300,width=500)
    dff2 = districtvotes_df[districtvotes_df['State Code'] == received_data]
    bar_chart2 = px.bar(dff2, x='District',
                         y='Votes',title=f'Number of district:{len(dff2)}',color= 'Party',color_discrete_map={
                             'Republician': 'rgb(178,24,43)',
                             'Democratic': 'rgb(33,102,172)'
                         },hover_data=['Candidate', 'Incumbent'],category_orders={'Party':['Republician','Democratic']})
    bar_chart2.update_layout(height=300,width=500)
    return win_statement,bar_chart,bar_chart2
    


app.run_server(mode='inline', port=8239)

In [ ]:
#with extra features
# instantiate a JupyterDash object instead of a Dash object
app = JupyterDash(__name__)
app.layout = html.Div([html.Div([html.H1(children='US House Election 2022', id="heading"),
                       html.Hr()]),
                      html.Div([html.P(children=f'Total seats: {len(df_plot)} | Republician won: {len(df_repu)} | Democratic won: {len(df_demo)}',
                             style={'font-weight': 'bold'}),
                                dcc.Graph(id='us-election-2022-fig', figure=fig,clickData={'points': [{'customdata': ['AL']}]}),
                                dcc.Graph(id='winning_candidates',figure=bar_chart3)],
                               style={'width': '70%', 'display': 'inline-block', 'padding': '0 20', 'float': 'left'}),
                       html.Div([html.P(id ='winning_statement',style={'font-weight': 'bold','font-size': '13px'}),
                                 dcc.Graph(id='winning-party-votes'),
                                 dcc.Graph(id='winning-districts-votes',clickData={'points': [{'customdata': ['Carl']}]}),
                                 dcc.Graph(id='other-candidates')],
                               style={'width': '29%', 'display': 'inline-block', 'padding': '0 20', 'float': 'right'})
                       
                        ])


@app.callback(Output('winning_statement','children'),
              Output('winning-party-votes','figure'),
              Output('winning-districts-votes','figure'),
              Input('us-election-2022-fig', 'clickData'))
def UsElection(received):
    if not received: 
        raise PreventUpdate
    received_data = received['points'][0]['customdata'][0]
    won_parties = final_df.loc[final_df['State Code'] == received_data,'Won Party'].values[0]
    win_statement = f'Winning Party: {won_parties}'
    dff = partyvotes_df[partyvotes_df['State Code'] == received_data]
    required_state = df.loc[df['State Code'] == received_data, 'State'].values[0]
    bar_chart = px.bar(dff, x='Parties',
                         y='Votes',title= f'State: {required_state}',color= 'Parties',hover_data=['color'],
                       color_discrete_map={'Republician': 'rgb(178,24,43)',
                                           'Democratic': 'rgb(33,102,172)',
                                           'Libertarian': 'rgb(253,219,199)',
                                           'Green': 'rgb (0,128,0)',
                                           'Independent': 'rgb(255,255,0)',
                                           'Other': 'rgb(26,211,243)'},
                       category_orders={'Parties':['Republician','Democratic']})
    bar_chart.update_layout(height=300,width=500)
    dff2 = districtvotes_df[districtvotes_df['State Code'] == received_data]
    bar_chart2 = px.bar(dff2, x='District',
                         y='Votes',title=f'Number of district:{len(dff2)}',color= 'Party',color_discrete_map={
                             'Republician': 'rgb(178,24,43)',
                             'Democratic': 'rgb(33,102,172)'
                         },hover_data=['Candidate', 'Incumbent'],category_orders={'Party':['Republician','Democratic']})
    bar_chart2.update_layout(height=300,width=500)
    
    
    return win_statement,bar_chart,bar_chart2

@app.callback(Output('other-candidates','figure'),
              Input('winning-districts-votes','clickData'))     

def District_Data(clickData):
    if not clickData:
        raise PreventUpdate
    candidate = clickData['points'][0]['customdata'][0]
    district = df.loc[df['Candidate'] == candidate,'District'].values[0]
    state = df.loc[df['Candidate'] == candidate,'State'].values[0]
    dff4 = df[df['State'] == state]
    dff4 = dff4[dff4['District'] == district]
    dff4 = dff4.replace({'Libertarian Party': 'Libertarian', 'Green Party': 'Green', 'Independent Party': 'Independent', 'Other Party': 'Other'})
    bar_chart4 = px.bar(dff4, x='Candidate', y='Votes', title=f'All candidates in {district} district of {state}',
                color='Party', color_discrete_map={'Republician': 'rgb(178,24,43)',
                                                   'Democratic': 'rgb(33,102,172)',
                                                   'Libertarian': 'rgb(253,219,199)',
                                                   'Green': 'rgb (0,128,0)',
                                                   'Independent': 'rgb(255,255,0)',
                                                   'Other': 'rgb(26,211,243)'},
                hover_data=['Candidate', 'Incumbent', 'Votes', 'Pct%'], category_orders={'Party': ['Republican', 'Democratic']})
    bar_chart4.update_layout(height=300, width=500)
    return bar_chart4

app.run_server(mode='inline', port=8239)